In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import pandas as pd
import numpy as np

import random
import re
import time
import traceback

- 006751 전체대학
- 126897 KU융합과학기술원
- 127662 KU혁신공유대학
- 105271 건축대학
- 105541 경영대학
- 103041 공과대학
- 100251 교무처
- 127772 국제대학
- 126940 국제처
- 122046 글로벌융합대학
- 126843 대학교육혁신원
- 102761 문과대학
- 127425 부동산과학원
- 104951 사범대학
- 127119 사회과학대학
- 103781 상경대학
- 126841 상허교양대학
- 126896 상허생명과학대학
- 104351 생명환경과학대학
- 105061 수의과대학
- 122045 예술디자인대학
- 105121 이과대학
- 127304 혁신공유대학(글로컬)

## 컬럼 정의
- 연도, 과목번호, 원어여부, 비고, 수강신청유의사항, 평가비율, 교재, 과제내용, 과제수, 주별강의계획서, 중간고사, 기말고사  
=> 전선 데이터셋과 비교하여 단과대학 컬럼이 없음
- 과제비율, 교재, 과제내용, 주별강의계획서, 중간고사, 기말고사 컬럼은 딕셔너리 형태로 데이터가 저장되어 있음
- 교재, 과제내용, 주별강의계획서 컬럼은 과목별 강의계획서 해당 표의 내용을 전부 가져온 것으로, 단순 참고용 

In [2]:
# 초기값
syllabus_list = []  # 각 강의계획서 내용이 데이터프레임 형태로 차례로 저장되는 빈 리스트 정의
debug_trial = 0     # 파싱에 실패하는 횟수

# ================================================================================================

# 크롬 옵션 설정
options = Options()
options.add_argument("--start-maximized")  # 창 최대화
options.add_argument("--incognito")  # 시크릿 모드로 실행
options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화

# 크롬 드라이버 사용
driver = webdriver.Chrome(options=options)
driver.get('https://sugang.konkuk.ac.kr/sugang/jsp/search/searchMainOuter.jsp')

# 메인 창 핸들 저장
main_window_handle = driver.current_window_handle

# 드롭다운 로딩 대기
time.sleep(2)

# 연도 반복
for year in ['2022', '2023', '2024']:  # 텍스트로 지정해야 합니다.
    print(f'Current Year: {year}')

    # 드롭다운 - 연도
    select_year = driver.find_element(By.NAME, 'pYear')
    Select(select_year).select_by_value(year)

    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))

    # 드롭다운 - 학기 (2학기만 확인)
    select_semester = driver.find_element(By.NAME, 'pTerm')
    Select(select_semester).select_by_value('B01012')

    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))

    # 드롭다운 - 이수구분
    select_type = driver.find_element(By.NAME, 'pPobt')
    Select(select_type).select_by_value('B04054')

    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))

    # 조회 버튼 클릭
    search_button = driver.find_element(By.CLASS_NAME, 'btn-sub')
    search_button.click()
    
    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))
    
    # 과목번호 - 클래스명이 'btn-sm btn-sub'인 모든 버튼 가져오기
    number_buttons = driver.find_elements(By.CLASS_NAME, 'btn-sm.btn-sub')

    # 원어 - aria-describedby="gridLecture_lang_type_nm" 속성을 가진 모든 <td> 요소 찾기
    lang_type = driver.find_elements(By.XPATH, '//td[@aria-describedby="gridLecture_lang_type_nm"]')
    
    # 비고 - aria-describedby="gridLecture_remk" 속성을 가진 모든 <td> 요소 찾기
    lecture_remk = driver.find_elements(By.XPATH, '//td[@aria-describedby="gridLecture_remk"]')
    
    # 적응 시간
    time.sleep(np.round(random.uniform(0.5, 1), 3))
    
    # 각 버튼 클릭 및 팝업 처리
    for row_idx, button in enumerate(number_buttons):
        # 과목번호 추출하기
        onclick_value = button.get_attribute('onclick')
        button_number = re.search(r"\('(\d+)'\)", onclick_value).group(1)  # 정규표현식으로 숫자 부분만 추출
        
        # 클릭할 강의계획서의 내용을 저장할 딕셔너리 정의
        syllabus_dict = {
            '연도': year,
            '과목번호': button_number,
            '원어여부': lang_type[row_idx].text,
            '비고': lecture_remk[row_idx].text
        }
        
        # 클릭할 버튼이 화면에 보이도록 스크롤
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(0.5)
        
        # 강의계획서 클릭
        button.click()
        
        # 팝업창 대기 (팝업이 뜨면 창 개수가 증가)
        WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
        
        # 팝업창 핸들 탐색 및 전환
        for handle in driver.window_handles:
            if handle != main_window_handle:
                driver.switch_to.window(handle)  # 팝업창으로 전환
                break
        
        # 팝업창에서 원하는 데이터 크롤링
        try:
            # 팝업창에서 텍스트가 모두 뜰 때까지 대기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'tbl-detail'))
            )
    
            # HTML 소스 가져오기
            page_source = driver.page_source

            # BeautifulSoup으로 HTML 파싱
            soup = BeautifulSoup(page_source, 'html.parser')

            # <table class="tbl-detail"> 안의 <td> 태그 텍스트 추출
            tables = soup.find_all('table', class_='tbl-detail')  # 모든 <table class="tbl-detail"> 요소 찾기
            
            # 모든 테이블의 <td> 태그 텍스트를 추출
            for table_idx, table in enumerate(tables, 1):  # 각 테이블에 대해 반복
                # 현재 테이블의 모든 <td> 태그 가져오기
                td_elements = table.find_all('td')
                td_texts = {idx: td.text.strip() for idx, td in enumerate(td_elements, 1)}  # 텍스트만 추출하고 양쪽 공백 제거

                # 강의계획서 내용이 존재하지 않는 경우 결측치로 처리
                if len(tables) == 1:
                    break
                    
                # 일반사항 테이블 - '수강신청 유의사항'만 추츨
                if table_idx == 1:
                    cleaned_text = td_texts[5].replace("-", "")  # 엑셀 표기상 오류를 막기 위해 텍스트에서 - 제거
                    syllabus_dict['수강신청유의사항'] = cleaned_text.strip()  # 양쪽 공백 제거

                # 과제비율 테이블 - 비율이 0보다 큰 항목들만 가져오기
                elif table_idx == 2:
                    rate_dict = {}

                    # td_texts를 4개씩 그룹으로 묶어 처리 (최종적으로 항목 이름, 비율만 추출)
                    for i in range(0, len(td_texts), 4):
                        rate = td_texts[i+2]
    
                        # 숫자인지 확인 후, 0보다 크면 딕셔너리에 추가
                        try:
                            rate_value = float(rate)
                            if rate_value > 0:
                                rate_dict[td_texts[i+1]] = rate_value
                        except ValueError:
                            pass  # 숫자가 아닌 경우 무시
                        
                        # 비율 합이 100이 되면 중단
                        if sum(rate_dict.values()) >= 100:
                            break

                    # 비율이 0보다 큰 항목들만 딕셔너리로 묶어 저장
                    syllabus_dict['과제비율'] = rate_dict

                # 강의교재 테이블
                elif table_idx == 3:
                    syllabus_dict['교재'] = td_texts if len(td_texts) > 1 else np.nan
                
                # 강의과제 테이블 - 과제내용, 과제 수
                elif table_idx == 4:
                    syllabus_dict['과제내용'] = td_texts if len(td_texts) > 1 else np.nan
                    syllabus_dict['과제수'] = round(len(td_texts)/3)  # 내용이 없을 경우 0

                # 주별강의계획서 테이블 - 주별강의계획서 전체내용, 중간고사 행, 기말고사 행
                else:
                    if len(td_texts) > 1:  # 내용이 없을 경우 입력하지 않음
                        syllabus_dict['주별강의계획서'] = td_texts
                        syllabus_dict['중간고사'] = {
                            '주제': td_texts[44],
                            '강의내용': td_texts[45],
                            '수업유형': td_texts[46],
                            '강의활동': td_texts[47]
                        }
                        syllabus_dict['기말고사'] = {
                            '주제': td_texts[92],
                            '강의내용': td_texts[93],
                            '수업유형': td_texts[94],
                            '강의활동': td_texts[95]
                        }
            
        except: 
            # 오류 메시지 출력
            print(f'{button_number} 과목 - HTML 파싱 실패')
            traceback.print_exc()  # 전체 스택 추적 출력

            # 팝업창 스크린샷 촬영
            debug_trial += 1
            driver.save_screenshot(f"debug_screenshot_{debug_trial}.png")

        # 강의계획서 업데이트
        syllabus_list.append(pd.DataFrame([syllabus_dict]))
        
        # 팝업창 닫기
        driver.close()
        driver.switch_to.window(main_window_handle)  # 메인 창으로 복귀

# 크롤링 완료
driver.quit()

# 모든 강의계획서를 데이터프레임으로 한번에 병합
result_df = pd.concat(syllabus_list, ignore_index=True)
print("크롤링이 모두 완료되었습니다!")

Current Year: 2022
Current Year: 2023
Current Year: 2024
크롤링이 모두 완료되었습니다!


In [3]:
result_df.head()

,연도,과목번호,원어여부,비고,수강신청유의사항,과제비율,교재,과제내용,과제수,주별강의계획서,중간고사,기말고사
0,2022,1501,영어,원어강의(영어),Regular attendance and participation is required.,"{'출석률': 10.0, '중간고사비율': 25.0, '기말고사비율': 25.0, ...","{1: '주교재', 2: 'Prism 4 Reading', 3: 'Jessica W...","{1: 'Midterm Presentation', 2: '20221014', 3: ...",2,"{1: '08/29 ~ 09/03', 2: 'Introduction', 3: 'Co...","{'주제': 'Midterm Exam', '강의내용': 'Midterm Exam',...","{'주제': 'Final Exam', '강의내용': 'Final Exam', '수업..."
1,2022,1502,,토론식 강의,본 강의는 토론식 수업으로 수업 중 일부가 토론 형태로 진행됩니다.\n★ 토론식 수...,"{'출석률': 15.0, '중간고사비율': 25.0, '기말고사비율': 30.0, ...","{1: '주교재', 2: '인간관계의 심리학 (젊은이를 위한)', 3: '권석만',...","{1: '서평(또는 독후감)', 2: '', 3: '', 4: '보물지도 만들기',...",2,"{1: '08/29 ~ 09/03', 2: 'Orientation', 3: '- 강...","{'주제': '중간고사', '강의내용': '- 중간고사', '수업유형': '', '...","{'주제': '기말고사', '강의내용': '- 기말고사', '수업유형': '', '..."
2,2022,1503,,,1. 이 강의는 수강생의 성실성이 요구됩니다. 신중히 수강신청하세요.\n2. 지각 ...,"{'출석률': 10.0, '중간고사비율': 35.0, '기말고사비율': 35.0, ...","{1: '참고문헌', 2: '논리는 나의 힘 (생각의 힘을 길러 주는 논리 학습의 ...","{1: '진리표에 관하여', 2: '', 3: '', 4: '벤다이어그램에 관하여'...",2,"{1: '08/29 ~ 09/03', 2: '강의 내용과 진행 소개', 3: '강의...","{'주제': '중간고사', '강의내용': '시험', '수업유형': '대면시험', '...","{'주제': '학기말고사', '강의내용': '시험', '수업유형': '대면시험', ..."
3,2022,1504,,,"본 교과목은 이러닝이므로, 시험, 퀴즈 시간이 사전에 정해져 있습니다. 이 시간이 ...","{'출석률': 20.0, '중간고사비율': 25.0, '기말고사비율': 25.0, ...","{1: '주교재', 2: '생활속에서 배우는 소비자교육', 3: '김시월', 4: ...","{1: '토론( 생각해보기)이 10회 있습니다.', 2: '', 3: '', 4: ...",3,"{1: '08/29 ~ 09/03', 2: '우리 과목을 소개합니다.', 3: '-...","{'주제': '중간고사', '강의내용': '중간고사(온라인); 10월 18일(화요일...","{'주제': '기말고사; 온라인', '강의내용': '기말고사(온라인); 12월 13..."
4,2022,1505,,,강의는 매주 조별 발표와 토론으로 이루어짐. 비대면일 경우 구두시험을 치르며 강의에...,"{'출석률': 10.0, '중간고사비율': 30.0, '기말고사비율': 30.0, ...","{1: '주교재', 2: '젊은 베르테르의 슬픔', 3: '요한 볼프강 폰 괴테',...","{1: '강의에서 다루는 작품과 유사한 우리나라 작품 찾아서 비교해보기', 2: '...",1,"{1: '08/29 ~ 09/03', 2: '오리엔테이션', 3: '강의 소개 및 ...","{'주제': '중간고사', '강의내용': '중간고사', '수업유형': '대면시험',...","{'주제': '기말고사', '강의내용': '기말고사', '수업유형': '대면시험',..."


In [4]:
result_df.tail()

,연도,과목번호,원어여부,비고,수강신청유의사항,과제비율,교재,과제내용,과제수,주별강의계획서,중간고사,기말고사
564,2024,1725,,,"본 수업은 영화(가디언즈 오브 갤럭시 3, 옥자, 베일리 어게인 등)라는 매개를 통...","{'출석률': 15.0, '기말고사비율': 30.0, '과제물비율': 30.0, '...","{1: '주교재', 2: '동물은 인간에게 무엇인가 (인간과 동물의 관계를 통찰하는...","{1: '', 2: '', 3: ''}",1,"{1: '09/02 ~ 09/07', 2: '오리엔테이션, 교과목 설명 및 수업방법...","{'주제': '중간고사', '강의내용': '강의에서 소개된 영화 외, 인간동물관계를...","{'주제': '기말고사', '강의내용': '강의록 기반 시험 예정', '수업유형':..."
565,2024,1729,,,본 수업은 이러닝으로 진행됩니다. 기말 고사 기간은 본 기말고사 이전 1주일전에 진...,"{'출석률': 15.0, '중간고사비율': 35.0, '기말고사비율': 35.0, ...","{1: '주교재', 2: '탄소중립과 에너지 법', 3: '정연덕', 4: '건국대...",NaN,0,"{1: '09/02 ~ 09/07', 2: '오리엔테이션', 3: '수업 전체 오리...","{'주제': '중간과제', '강의내용': '중간과제', '수업유형': '', '강의...","{'주제': '기말고사', '강의내용': '기말고사', '수업유형': '', '강의..."
566,2024,1731,,,본 수업은 이캠퍼스를 통해서 많은 소통을 합니다. 상세사항은 이캠퍼스를 통해 공지합...,"{'출석률': 20.0, '중간고사비율': 30.0, '기말고사비율': 30.0, ...","{1: '주교재', 2: '인공지능시대 1권 /2권', 3: '박재영', 4: '형...",{1: '주차별 주제와 관련한 학술논문 1개를 찾아서 1단락요약/3단락 비평 (각 ...,1,"{1: '09/02 ~ 09/07', 2: 'Course Introduction/A...","{'주제': '중간고사', '강의내용': '중간고사', '수업유형': '온라인실시간...","{'주제': '기말고사', '강의내용': '기말고사', '수업유형': '온라인실시간..."
567,2024,1733,,,1. 전문적 영화 감상과 비평에 필요한 가장 기본적인 영화학 개론을 소개하는 강의입...,"{'출석률': 15.0, '중간고사비율': 35.0, '기말고사비율': 35.0, ...","{1: '참고문헌', 2: '세계영화사(Film History) (An Introd...","{1: '1주차 강의에서 자세하게 소개할 예정입니다.', 2: '20241128',...",1,"{1: '09/02 ~ 09/07', 2: '강의 소개', 3: '학기 중 다루에 ...","{'주제': '중간 필기 시험', '강의내용': '중간 필기 시험', '수업유형':...","{'주제': '기말 시험', '강의내용': '기말 필기 시험 실시', '수업유형':..."
568,2024,1734,,교환학생만 수강 가능,"Since this class is a Korean class, the teache...","{'출석률': 15.0, '중간고사비율': 35.0, '기말고사비율': 35.0, ...","{1: '주교재', 2: '함께 하는 건국 한국어1', 3: '손재은', 4: '건...",NaN,0,"{1: '09/02 ~ 09/07', 2: 'Orientation', 3: 'In ...","{'주제': 'Mid-test', '강의내용': 'Mid-test', '수업유형':...","{'주제': 'Final-test', '강의내용': 'Final-test', '수업..."


In [5]:
result_df.shape

(569, 12)

In [6]:
result_df.to_csv('강의계획서_심교.csv',index=False,encoding='cp949')